In [ ]:
# set up package roots

import matplotlib.pyplot as plt

%env DRAKE_RESOURCE_ROOT = '/drake/share'

import sys
sys.path.append('/Users/lawrenceli/Documents/mit/6832/project/underactuated/src')
sys.path.append('/Users/lawrenceli/Documents/mit/6832/project/drake-env/lib/python2.7/site-packages/')

Let's first try the controller's trajectory optimization method for the passive compass gait in steady state, as a sanity check.

In [ ]:
%load_ext autoreload
%autoreload 2

from compass_gait_sim import ComputeTrajectory
import numpy as np

# steady state conditions

passive = True
equal_time_steps = True

if passive:
    # passive compass gait
    slope = None # default slope
    state_initial = np.array([-0.219, 0.324, 1.093, 0.376])
    state_final = np.array([0.324, -0.219, 1.496, 1.81])
else:
    # actuated compass gait on flat ground
    slope = 0
    state_initial = np.array([0.2884, -0.2884, -1.1235, -0.2830])
    state_final = np.array([-0.2884, 0.2884, -1.6009, -1.9762])

min_time = 1
max_time = 5
    
states, u, time_array = ComputeTrajectory(state_initial, state_final, min_time, max_time, 
                                          DMOC=True, 
                                          slope=slope, 
                                          equal_time_steps = equal_time_steps)

import matplotlib.pyplot as plt
plt.plot(u)
plt.savefig('figs/u_equal_time_steps=%r' % equal_time_steps)
plt.show()

plt.plot(np.diff(time_array))
plt.savefig('figs/time_array_equal_time_steps=%r' % equal_time_steps)
plt.show()

In [ ]:
%load_ext autoreload
%autoreload 2

from compass_gait_sim import Simulate2dCompassGait
minimal_state_freq = 1./100
viz, state_log, minimal_state_log = Simulate2dCompassGait(10, passive = True, passive_controller = True, 
                                                         minimal_state_freq = minimal_state_freq)

In [ ]:
%load_ext autoreload
%autoreload 2

from compass_gait_sim import ComputeTrajectory
import numpy as np

states = minimal_state_log.data().T
resets = []
for i in range(len(states)-1):
    if states[i,0] * states[i+1,0] < 0 and abs(states[i,0]) > 0.01: # found a reset state
        resets.append(i)
ss_states = states[resets[-2]+1:resets[-1]]
u_traj = np.zeros(len(ss_states))
time_step = minimal_state_freq
time_array = np.arange(len(ss_states)+1) * time_step
test_traj = (ss_states[:,:2], u_traj, time_array)

state_initial = ss_states[0]
state_final = ss_states[-1]
min_time = 1
max_time = 10

if True:
    plt.plot(ss_states[:-1,2], color='orange')
    plt.scatter(np.arange(len(ss_states)-1), np.diff(ss_states[:,0]) / time_step, s=10)
    plt.savefig('figs/discrete_differences_freq=%d' % int(1./minimal_state_freq))
    plt.show()
    #for i in range(len(ss_states)-1):
    #    print ss_states[i,2], (ss_states[i+1,0]-ss_states[i,0])/time_step

ComputeTrajectory(state_initial, state_final, min_time, max_time, test_traj, DMOC=True)


In [ ]:
import matplotlib.pyplot as plt
ani = viz.animate(state_log, 10, repeat=True)
plt.close(viz.fig)
HTML(ani.to_html5_video())